In [1]:
import os 
os.chdir('../')

In [2]:
from backbones.sana import SANA

# 사용 예
model = SANA()
print(model)

/home/scpark/miniconda3/envs/sana/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/scpark/miniconda3/envs/sana/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/scpark/miniconda3/envs/sana/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
Loading pipeline components...:  40%

Loading pipeline components...: 100%|██████████| 5/5 [00:01<00:00,  4.80it/s]


In [3]:
import matplotlib.pyplot as plt

def show_compare(imgs):
    fig, axes = plt.subplots(1, len(imgs), figsize=(18, 10))
    for i in range(len(imgs)):
        axes[i].imshow(imgs[i]); axes[i].axis('off')
    plt.tight_layout()
    plt.show()


In [4]:
from solvers.euler_solver import Euler_Solver
from solvers.dpm_solver import DPM_Solver
from solvers.adams_solver import Adams_Solver
from solvers.adams_dual_solver import Adams_Dual_Solver

pos_texts = ["A serene twilight view of a traditional Korean hanok village nestled between misty mountain slopes: curved midnight-blue tiled eaves, softly glowing paper lanterns swaying in the breeze; ancient pine trees arching over stone pathways; intricate wooden lattice windows casting delicate shadows; a lone scholar in flowing hanbok practicing calligraphy beside a koi pond with lotus petals drifting on the water; cinematic 8K ultra-realism with dynamic volumetric moonlight filtering through morning mist; painterly strokes blending classical Joseon-era ink wash with modern hyperrealism; shot on RED Monstro 8K, 50 mm f/1.2 lens; subtle film grain; maximum fidelity; emotional atmosphere."]
neg_texts = ["lowres, bad anatomy, deformed, blurry, pixelated, oversaturated, underexposed, overexposed, artifact, jpeg artifacts, watermark, text, logo, extra limbs, mutated hands, unnatural colors, noisy background, out of focus, poor composition, cultural clichés, stereotype exaggeration, flat lighting, glitch"]

prompts = [
    "A photorealistic portrait of an elderly Asian woman with silver hair and deep wrinkles, soft studio lighting, 8K resolution",
    "A fantasy landscape with towering crystal spires, floating islands, and a purple sunset sky, in the style of studio ghibli",
    "A futuristic cyberpunk street market at night, neon signs in Japanese kanji, wet pavement reflecting lights, cinematic atmosphere",
    "A detailed illustration of a steampunk airship flying above the clouds, brass gears and pipes visible, watercolor painting style",
    "An isometric view of a cozy medieval tavern interior, wooden beams, roaring fireplace, characters playing dice",
    "A macro shot of a dew-covered spider web glistening in the early morning light, extreme close-up, bokeh background",
    "An abstract expressionist painting of swirling colors and shapes, evoking emotion rather than form, thick brush strokes",
    "A cartoon-style illustration of a cheerful corgi wearing a detective hat and holding a magnifying glass, vibrant colors",
    "A high-contrast black-and-white film noir scene: a trench-coated detective under a streetlamp in the rain, moody shadows",
    "An ultra-wide aerial view of a winding mountain road through autumn forest, golden leaves, dramatic clouds overhead"
]

for prompt in prompts:
    model_fn, noise_schedule, latents = model.get_model_fn(pos_conds=[prompt], neg_conds=neg_texts, guidance_scale=4.5, seeds=[42])
    flow_shift = 3.0

    solver = Euler_Solver(model_fn, noise_schedule, algorithm_type='vector_prediction')
    latent_samples = solver.sample(latents, steps=100, skip_type='time_uniform_flow', flow_shift=flow_shift)
    euler_sample = model.decode_vae(latent_samples)[0]

    solver = Adams_Solver(model_fn, noise_schedule, algorithm_type='data_prediction')
    latent_samples = solver.sample(latents, steps=10, order=2, skip_type="time_uniform_flow", flow_shift=flow_shift, use_corrector=False)
    adams1_sample = model.decode_vae(latent_samples)[0]

    solver = Adams_Solver(model_fn, noise_schedule, algorithm_type='vector_prediction')
    latent_samples = solver.sample(latents, steps=10, order=2, skip_type="time_uniform_flow", flow_shift=flow_shift, use_corrector=True)
    adams2_sample = model.decode_vae(latent_samples)[0]

    solver = Adams_Dual_Solver(model_fn, noise_schedule, algorithm_type='dual_prediction')
    latent_samples = solver.sample(latents, steps=10, order=2, skip_type="time_uniform_flow", flow_shift=flow_shift, use_corrector=False)
    adams3_sample = model.decode_vae(latent_samples)[0]

    solver = Adams_Dual_Solver(model_fn, noise_schedule, algorithm_type='dual_prediction')
    latent_samples = solver.sample(latents, steps=10, order=2, skip_type="time_uniform_flow", flow_shift=flow_shift, use_corrector=True)
    adams4_sample = model.decode_vae(latent_samples)[0]

    show_compare([euler_sample, adams1_sample, adams2_sample, adams3_sample, adams4_sample])